In [1]:
import sqlite3

In [2]:
conn = sqlite3.connect('test.db')
conn

In [14]:
import pathlib
conn = sqlite3.connect(pathlib.Path('test.db'))
conn

In [16]:
with open(pathlib.Path('test.db'), 'rb') as f:
    print(f.read())

b''


In [20]:
import datetime
now=datetime.datetime.now()
strtime = now.strftime('%Y-%m-%d %H:%M:%S')
strtime

'2022-07-07 13:39:14'

In [89]:
from nonebot.log import logger
import os
import sqlite3
from sqlite3 import Connection
from pathlib import Path
import datetime

from sqlalchemy import func

DB_PATH: Path = Path("")


def try_except_ensure(func):
    def new_func(*args) -> Optional[str, list[str]]:
        try:
            return func(*args)
        except Exception as e:
            def __(e : Exception, *args) -> str: 
                return f'出错了:{type(e)}|{str(e)}'  
            return __(e, *args)
    return new_func

def open_table(path: Path) -> Connection:
    if not path.exists():
        conn = sqlite3.connect(path)
        cursor = conn.cursor()
        cursor.execute('''
            CREATE TABLE MEMO(
            ID INTEGER PRIMARY KEY AUTOINCREMENT,
            DATA           TEXT     ,
            USER           TEXT     ,
            TIME           TEXT     );
        '''.strip())
    else:
        conn = sqlite3.connect(path)

    return conn

@try_except_ensure
def memo_add(group_id: str, user_id: str, data: str) -> str:
    '''把数据存进数据库, 返回提示信息'''
    
    conn = open_table(DB_PATH / (group_id + '.db'))
    cursor = conn.cursor()

    nowtime=datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')

    cursor.execute(f'''
        INSERT INTO MEMO (USER, TIME, DATA)
        VALUES ("{user_id}", "{nowtime}", "{data}");
    '''.strip())

    conn.commit()
    conn.close()

    return '大概写好了~'

@try_except_ensure
def memo_see(group_id: str) -> list[str]:
    '''以字符串列表形式返回数据库信息'''
    conn = open_table(DB_PATH / (group_id + '.db'))
    cursor = conn.cursor()
    cursor.execute('''
        SELECT ID, USER, TIME, DATA
        from MEMO;
    ''')

    print(cursor.fetchall())

    lst, text = [], ''
    for mid, userid, datatime, data in cursor.fetchall():
        print(mid, userid, datatime, data)
        text += f'{mid}. {data}\n 由{userid}于{datatime}写入\n'
        if len(text) >= 512:
            lst.append(text)
            text = ''
    if text != '': lst.append(text)

    return lst

@try_except_ensure
def memo_del(group_id: str, index: str) -> str:
    conn = open_table(DB_PATH / (group_id + '.db'))
    cursor = conn.cursor()

    cursor.execute(f'''
        SELECT ID 
        FROM MEMO 
        WHERE ID = "{int(index)}";
    ''')
    if cursor.fetchall() == []:
        return '序号错误~'

    cursor.execute(f'''
        DELETE 
        FROM MEMO
        WHERE ID = "{int(index)}";
    ''')
    conn.commit()

    return '大概删了~'


[(3, 'asdf', '2022-07-07 14:57:28', '8'), (4, 'asdf', '2022-07-07 14:57:28', '7'), (5, 'asdf', '2022-07-07 14:57:33', '8'), (6, 'asdf', '2022-07-07 14:57:33', '7'), (7, 'asdf', '2022-07-07 14:57:43', '8'), (8, 'asdf', '2022-07-07 14:57:43', '7'), (9, 'asdf', '2022-07-07 14:57:49', '8'), (10, 'asdf', '2022-07-07 14:57:49', '7'), (11, 'asdf', '2022-07-07 15:00:09', '8'), (12, 'asdf', '2022-07-07 15:00:09', '7'), (13, 'asdf', '2022-07-07 15:02:44', '8'), (15, 'asdf', '2022-07-07 15:02:47', '8'), (16, 'asdf', '2022-07-07 15:02:47', '7'), (17, 'asdf', '2022-07-07 15:02:50', '8'), (18, 'asdf', '2022-07-07 15:02:50', '7'), (19, 'asdf', '2022-07-07 15:02:55', '8'), (20, 'asdf', '2022-07-07 15:02:55', '7'), (21, 'asdf', '2022-07-07 15:03:01', '8'), (22, 'asdf', '2022-07-07 15:03:01', '7'), (23, 'asdf', '2022-07-07 15:03:23', '8'), (24, 'asdf', '2022-07-07 15:03:23', '7'), (25, 'asdf', '2022-07-07 15:03:33', '8'), (26, 'asdf', '2022-07-07 15:03:33', '7'), (27, 'asdf', '2022-07-07 15:03:46', '8')

[]

In [ ]:
def try_except_ensure1(func):
    def new_func(*args):
        try:
            return func(*args)
        except Exception as e:
            def __(e : Exception, *args) -> str: 
                return f'出错了:{type(e)}\n{str(e)}'
            return __(e, *args)
    return new_func

@try_except_ensure1

In [5]:
Default_Mod = 998244353

def int_size_ensure(func, mod : int = Default_Mod):
    return lambda *args: func(*tuple(int(each) % mod for each in args))

def try_except_ensure(func):
    def new_func(*args):
        try:
            return func(*args)
        except Exception as e:
            async def __(e : Exception, *args) -> str: 
                return f'出错了:{type(e)}\n{str(e)}'
            return __(e, *args)
    return new_func

@try_except_ensure
@int_size_ensure
def gcd(a : int, b : int) -> int:
    '''求a和b的最大公因数'''
    return a if not b else gcd(b, a % b)

@try_except_ensure
@int_size_ensure
async def lcm(a : int, b : int) -> int:
    '''求a和b的最小公倍数'''
    return a // await gcd(a, b) * b

@try_except_ensure
@int_size_ensure
async def power(x : int, p : int, mod : int = Default_Mod) -> int:
    '''求x的p次幂%mod'''
    base, ans = x, 1
    while p >= 1:
        if p & 1: ans = ans * base % mod
        base = base ** 2 % mod
        p >>= 1
    return ans

@try_except_ensure
@int_size_ensure
async def exgcd(a : int, b : int) -> int:
    '''扩展欧几里得'''
    if b == 0: return 1, 0, a
    x, y, ret = await exgcd(b, a % b)
    return y, (x - (a // b) * y), ret

@try_except_ensure
@int_size_ensure
async def inverse_Fermat(x : int, p : int) -> int:
    '''求xmodp的逆元, 费马小定理, 只给答案, 不验证数据合法性'''
    return await power(x, p - 2, p)

@try_except_ensure
@int_size_ensure
async def inverse_exgcd(x : int, p : int) -> int:
    '''求xmodp的逆元, 扩展欧几里得'''
    x, y, ret = await exgcd(x, p)
    return x % p if ret == 1 else -0x7fffffff

@try_except_ensure
@int_size_ensure
async def isprime(x : int) -> bool:
    '''判断x是否为素数'''
    if x <= 1: return False
    for i in range(2, int(x ** 0.5) + 1):
        if not x % i: 
            return False
    return True



In [30]:
a = 'asdf'
a = [a]
a

['asdf']